First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

Initalize variables

In [46]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [47]:
data = pd.read_csv(f'{path}/data/ltw.csv', encoding = 'UTF-8-SIG', sep=',')
#data = data.set_index(['AGS', 'year'])
data.head(10)

,AGS,GEN,year,Land,post_2014,Union,SPD,Grüne,FDP,Linke,...,treated_20,treated_30,treated_50,treated_100,treated_0*post_2014,treated_10*post_2014,treated_20*post_2014,treated_30*post_2014,treated_50*post_2014,treated_100*post_2014
0,3101000,Braunschweig,2008.0,Lower-Saxony,0.0,36.861503,31.245464,11.023763,8.087549,9.549529,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3101000,Braunschweig,2013.0,Lower-Saxony,0.0,29.666789,33.671272,18.608499,8.250600,4.246117,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3101000,Braunschweig,2017.0,Lower-Saxony,1.0,26.342641,37.604703,11.862375,7.904115,6.741527,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3102000,Salzgitter,2008.0,Lower-Saxony,0.0,37.476063,39.268958,4.291938,5.740138,8.828035,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3102000,Salzgitter,2013.0,Lower-Saxony,0.0,30.880127,44.075623,9.252751,6.053921,4.135136,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3102000,Salzgitter,2017.0,Lower-Saxony,1.0,24.445995,45.203436,4.266019,5.319891,4.862732,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,3103000,Wolfsburg,2008.0,Lower-Saxony,0.0,43.121141,31.515784,6.509262,6.691886,7.863727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3103000,Wolfsburg,2013.0,Lower-Saxony,0.0,37.629895,35.812910,10.901908,7.342017,3.210075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3103000,Wolfsburg,2017.0,Lower-Saxony,1.0,29.942107,43.211115,5.754535,6.092242,4.525280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3151001,Adenbüttel,2008.0,Lower-Saxony,0.0,47.568209,30.960854,6.287070,7.117438,4.863582,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


View summary statistics

In [48]:
columns = ['AGS', 'year', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'treated_0', 'treated_50', 'treated_100']
data.describe().to_csv(f'{path}/tables/ltw/ltw_descriptives.csv', sep=';', encoding='utf-8-sig', decimal=',', columns=columns, float_format='%.2f')
data.describe()

,AGS,year,post_2014,Union,SPD,Grüne,FDP,Linke,Andere,treated_0,...,treated_20,treated_30,treated_50,treated_100,treated_0*post_2014,treated_10*post_2014,treated_20*post_2014,treated_30*post_2014,treated_50*post_2014,treated_100*post_2014
count,1.373000e+04,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,...,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000
mean,8.642552e+06,2013.027749,0.377130,41.831258,19.779630,8.472463,6.330580,6.857527,16.779390,0.065841,...,0.222870,0.289585,0.413911,0.674290,0.024035,0.057538,0.086453,0.111799,0.160379,0.264312
std,4.055690e+06,3.876644,0.484686,11.000669,11.491222,4.984088,3.866734,8.580106,10.549009,0.248013,...,0.416187,0.453586,0.492551,0.468657,0.153163,0.232877,0.281042,0.315130,0.366970,0.440982
min,3.101000e+06,2008.000000,0.000000,5.797101,0.000000,0.000000,0.000000,0.202429,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.558013e+06,2009.000000,0.000000,34.130754,10.291080,4.761905,3.389831,2.152034,7.760594,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.271125e+06,2013.000000,0.000000,41.245968,17.309479,7.434524,5.692517,3.488646,15.630401,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.678176e+06,2017.000000,1.000000,48.837961,28.285260,11.111111,8.254934,5.676469,24.031360,0.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.607706e+07,2019.000000,1.000000,85.300668,58.673469,38.003901,55.555556,45.535714,71.186441,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Calculate average vote shares in all years by treated 50

In [49]:
data.head()

,AGS,GEN,year,Land,post_2014,Union,SPD,Grüne,FDP,Linke,...,treated_20,treated_30,treated_50,treated_100,treated_0*post_2014,treated_10*post_2014,treated_20*post_2014,treated_30*post_2014,treated_50*post_2014,treated_100*post_2014
0,3101000,Braunschweig,2008.0,Lower-Saxony,0.0,36.861503,31.245464,11.023763,8.087549,9.549529,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3101000,Braunschweig,2013.0,Lower-Saxony,0.0,29.666789,33.671272,18.608499,8.250600,4.246117,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3101000,Braunschweig,2017.0,Lower-Saxony,1.0,26.342641,37.604703,11.862375,7.904115,6.741527,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3102000,Salzgitter,2008.0,Lower-Saxony,0.0,37.476063,39.268958,4.291938,5.740138,8.828035,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3102000,Salzgitter,2013.0,Lower-Saxony,0.0,30.880127,44.075623,9.252751,6.053921,4.135136,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
ltw_avg = data[['Land', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'treated_50', 'year']].groupby(['Land', 'treated_50', 'year'], as_index = False).mean()
ltw_avg.head()

,Land,treated_50,year,Union,SPD,FDP,Linke,Grüne,Andere
0,Bavaria,0.0,2008.0,46.903057,13.961173,7.219590,3.623138,7.899087,20.509050
1,Bavaria,0.0,2013.0,52.856762,15.427911,2.727186,1.597639,6.829502,20.667210
2,Bavaria,0.0,2018.0,41.105682,7.285103,4.186585,2.280242,13.278892,31.974788
3,Bavaria,1.0,2008.0,48.929358,16.773799,6.000053,4.249929,6.533108,17.631132
4,Bavaria,1.0,2013.0,51.155339,18.662338,2.457471,2.109036,6.685103,19.042584


For each state, plot the average vote shares over years for treated and untreated municipalities

In [51]:
states = ['Bavaria', 'Hesse', 'NRW', 'Lower-Saxony', 'Thuringia']
for state in states:
    # keep only obs. corresponding to state
    plot_data = ltw_avg[ltw_avg['Land'] == state]
    # get years in a list
    years = plot_data.year.unique().astype(int).tolist()
    # split into treated and untreated
    treated = plot_data[plot_data['treated_50'] == 1]
    untreated = plot_data[plot_data['treated_50'] == 0]
    # initialize figure 
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), dpi=300)
    #set x labels
    plt.setp(axes, xticks=years)
    # title for entire figure
    fig.suptitle(f'Average Vote Shares \n {state} State Parliament', fontsize=20)
    # edit subplots
    axes[0, 0].set_title('Union', fontsize=12)
    axes[0, 0].set_ylabel('% Votes', fontsize=10)
    axes[0, 0].plot(treated['year'], treated['Union'], color='#000000', marker = 'o', label='Treated')
    axes[0, 0].plot(untreated['year'], untreated['Union'], color='#7f7f7f', marker = 'o', label='Untreated')
    axes[0, 0].legend(loc='lower left')
    axes[0, 0].axvline(x=2013, color='red', linestyle='--')

    axes[0, 1].set_title('SPD', fontsize=12)
    axes[0, 1].set_ylabel('% Votes', fontsize=10)
    axes[0, 1].plot(treated['year'], treated['SPD'], color='#ff1100', marker = 'o', label='Treated')
    axes[0, 1].plot(untreated['year'], untreated['SPD'], color='#ff7066', marker = 'o', label='Untreated')
    axes[0, 1].legend(loc='lower left')
    axes[0, 1].axvline(x=2013, color='red', linestyle='--')

    axes[1, 0].set_title('FDP', fontsize=12)
    axes[1, 0].set_ylabel('% Votes', fontsize=10)
    axes[1, 0].plot(treated['year'], treated['FDP'], color='#ffed00', marker = 'o', label='Treated')
    axes[1, 0].plot(untreated['year'], untreated['FDP'], color='#fff799', marker = 'o', label='Untreated')
    axes[1, 0].legend(loc='upper left')
    axes[1, 0].axvline(x=2013, color='red', linestyle='--')

    axes[1, 1].set_title('Linke', fontsize=12)
    axes[1, 1].set_ylabel('% Votes', fontsize=10)
    axes[1, 1].plot(treated['year'], treated['Linke'], color='#cc0055', marker = 'o', label='Treated')
    axes[1, 1].plot(untreated['year'], untreated['Linke'], color='#e06699', marker = 'o', label='Untreated')
    axes[1, 1].legend(loc='upper left')
    axes[1, 1].axvline(x=2013, color='red', linestyle='--')

    axes[2, 0].set_title('Grüne', fontsize=12)
    axes[2, 0].set_ylabel('% Votes', fontsize=10)
    axes[2, 0].plot(treated['year'], treated['Grüne'], color='#3a6e23', marker = 'o', label='Treated')
    axes[2, 0].plot(untreated['year'], untreated['Grüne'], color='#62bb3b', marker = 'o', label='Untreated')
    axes[2, 0].legend(loc='upper left')
    axes[2, 0].axvline(x=2013, color='red', linestyle='--')

    axes[2, 1].set_title('Andere', fontsize=12)
    axes[2, 1].set_ylabel('% Votes', fontsize=10)
    axes[2, 1].plot(treated['year'], treated['Andere'], color='darkgrey', marker = 'o', label='Treated')
    axes[2, 1].plot(untreated['year'], untreated['Andere'], color='lightgrey', marker = 'o', label='Untreated')
    axes[2, 1].legend(loc='upper left')
    axes[2, 1].axvline(x=2013, color='red', linestyle='--')

    # Save figure
    fig.savefig(f'{path}/figures/ltw/ltw_{state}_vote_shares.png')
    plt.close()

Same plot for presentations

In [52]:
states = ['Bavaria', 'Hesse', 'NRW', 'Lower-Saxony', 'Thuringia']
for state in states:
    # keep only obs. corresponding to state
    plot_data = ltw_avg[ltw_avg['Land'] == state]
    # get years in a list
    years = plot_data.year.unique().astype(int).tolist()
    # split into treated and untreated
    treated = plot_data[plot_data['treated_50'] == 1]
    untreated = plot_data[plot_data['treated_50'] == 0]
    # initialize figure 
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10), dpi=300)
    #set x labels
    plt.setp(axes, xticks=years)
    # title for entire figure
    fig.suptitle(f'Average Vote Shares \n {state} State Parliament', fontsize=20)
    # edit subplots
    axes[0, 0].set_title('Union', fontsize=12)
    axes[0, 0].set_ylabel('% Votes', fontsize=10)
    axes[0, 0].plot(treated['year'], treated['Union'], color='#000000', marker = 'o', label='Treated')
    axes[0, 0].plot(untreated['year'], untreated['Union'], color='#7f7f7f', marker = 'o', label='Untreated')
    axes[0, 0].legend(loc='lower left')
    axes[0, 0].axvline(x=2013, color='red', linestyle='--')

    axes[0, 1].set_title('SPD', fontsize=12)
    axes[0, 1].set_ylabel('% Votes', fontsize=10)
    axes[0, 1].plot(treated['year'], treated['SPD'], color='#ff1100', marker = 'o', label='Treated')
    axes[0, 1].plot(untreated['year'], untreated['SPD'], color='#ff7066', marker = 'o', label='Untreated')
    axes[0, 1].legend(loc='lower left')
    axes[0, 1].axvline(x=2013, color='red', linestyle='--')

    axes[0, 2].set_title('FDP', fontsize=12)
    axes[0, 2].set_ylabel('% Votes', fontsize=10)
    axes[0, 2].plot(treated['year'], treated['FDP'], color='#ffed00', marker = 'o', label='Treated')
    axes[0, 2].plot(untreated['year'], untreated['FDP'], color='#fff799', marker = 'o', label='Untreated')
    axes[0, 2].legend(loc='upper left')
    axes[0, 2].axvline(x=2013, color='red', linestyle='--')

    axes[1, 0].set_title('Linke', fontsize=12)
    axes[1, 0].set_ylabel('% Votes', fontsize=10)
    axes[1, 0].plot(treated['year'], treated['Linke'], color='#cc0055', marker = 'o', label='Treated')
    axes[1, 0].plot(untreated['year'], untreated['Linke'], color='#e06699', marker = 'o', label='Untreated')
    axes[1, 0].legend(loc='upper left')
    axes[1, 0].axvline(x=2013, color='red', linestyle='--')

    axes[1, 1].set_title('Grüne', fontsize=12)
    axes[1, 1].set_ylabel('% Votes', fontsize=10)
    axes[1, 1].plot(treated['year'], treated['Grüne'], color='#3a6e23', marker = 'o', label='Treated')
    axes[1, 1].plot(untreated['year'], untreated['Grüne'], color='#62bb3b', marker = 'o', label='Untreated')
    axes[1, 1].legend(loc='upper left')
    axes[1, 1].axvline(x=2013, color='red', linestyle='--')

    axes[1, 2].set_title('Andere', fontsize=12)
    axes[1, 2].set_ylabel('% Votes', fontsize=10)
    axes[1, 2].plot(treated['year'], treated['Andere'], color='darkgrey', marker = 'o', label='Treated')
    axes[1, 2].plot(untreated['year'], untreated['Andere'], color='lightgrey', marker = 'o', label='Untreated')
    axes[1, 2].legend(loc='upper left')
    axes[1, 2].axvline(x=2013, color='red', linestyle='--')

    # Save figure
    fig.savefig(f'{path}/figures/ltw/ltw_{state}_vote_shares_pres.png')
    plt.close()